In [1]:
import pandas as pd
import requests

from api.data import (store_database_for_eys_gene,
                      parse_lovd,
                      LOVD_PATH,
                      set_lovd_dtypes,
                      request_clinvar_api_data,
                      get_variant_ids_from_clinvar_name_api,
                      )
from api.data import save_lovd_as_vcf

pd.options.display.max_columns = 0

In [ ]:
store_database_for_eys_gene("lovd", override=False)

In [ ]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")

In [ ]:
for i in data:
    print(i)
    display(data[i])

In [ ]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

In [ ]:
save_lovd_as_vcf(data["Variants_On_Genome"], "./lovd.vcf")

In [ ]:
from subprocess import Popen

process = Popen("spliceai -I ./lovd.vcf -O ./lovd_output.vcf -R ../tools/spliceai/hg38.fa -A grch38".split())
process.wait()

In [ ]:
from api.tools import get_revel_scores

chromosome = 6
position = 65655758

results = get_revel_scores(chromosome, position)

display(results)

In [2]:
variation_ids = '148002'

frames = request_clinvar_api_data(variation_ids)

display(frames)

,uid,obj_type,accession,accession_version,title,record_status,gene_sort,chr_sort,location_sort,variation_set_name,variation_set_id,molecular_consequence_list,protein_change,fda_recognized_database,supporting_submissions_scv,supporting_submissions_rcv,germline_classification_description,germline_classification_last_evaluated,germline_classification_review_status,germline_classification_fda_recognized_database,clinical_impact_classification_description,clinical_impact_classification_last_evaluated,clinical_impact_classification_review_status,clinical_impact_classification_fda_recognized_database,clinical_impact_classification_trait_set,oncogenicity_classification_description,oncogenicity_classification_last_evaluated,oncogenicity_classification_review_status,oncogenicity_classification_fda_recognized_database,oncogenicity_classification_trait_set,variation_set_0_measure_id,variation_set_0_variation_name,variation_set_0_cdna_change,variation_set_0_aliases,variation_set_0_variant_type,variation_set_0_canonical_spdi,germline_set_0_trait_name
0,148002,copy number gain,VCV000148002,VCV000148002.,GRCh38/hg38 6p12.1-q12(chr6:53931543-68149750)x3,,BAG2,06,00000000000053931543,,,[],,,[SCV000177291],[RCV000137097],Pathogenic,2010/12/22 00:00,no assertion criteria provided,,,1/01/01 00:00,,,[],,1/01/01 00:00,,,[],157753,GRCh38/hg38 6p12.1-q12(chr6:53931543-68149750)x3,GRCh38/hg38 6p12.1-q12(chr6:53931543-68149750)x3,[],copy number gain,,See cases


In [ ]:
clinvar_data = pd.read_csv("C:\\Users\\Kajus\\Desktop\\clinvar_results.txt", sep='\t')

display(clinvar_data)

Explanation of whats happening in the code below:

Function to get all the ids from a gene name:
```python
get_variant_ids_from_clinvar_name_api(name: str, count: int)
```

function gets the ids from the clinvar api, the name is the gene name and the count is the maximum number of ids to get (api's limit is 500)

function returns a dictionary with the count and the list of ids:

```json
{
    'count': int,
    'idlist': List[str]
}
```

if the count is greater than the api's limit, the function will split the list of ids into smaller lists of 500 and then request the data from the api in chunks of 500 ids:

```python
id_lists = [id_list[i:i + max] for i in range(0, size, max)]
```

then the function will request the data from the api and concatenate the dataframes into a single dataframe:

```python
frames = request_clinvar_api_data(join)
variations = pd.concat([variations, frames], ignore_index=True)
```

The variant extraction function contains a lot of nested lists and dictionaries, so the function will flatten the data and then concatenate the dataframes into a single dataframe

**NOTE**

> joining function may have been implemented wrong due to the waiting time of the api.


In [ ]:
import pandas as pd

variations = pd.DataFrame()

max = 500
name = "EYS"
count = 2147483647

id_array = get_variant_ids_from_clinvar_name_api(name, count)
size = int(id_array['count'])
id_list = id_array['idlist']

id_lists = [id_list[i:i + max] for i in range(0, size, max)]

track = 0
for lists in id_lists:
    join = ",".join(lists)
    frame = request_clinvar_api_data(join)
    
    variations = pd.concat([variations, frame], ignore_index=True)
    
    print(f"{track + 1}/{len(id_lists)}")
    track += 1

display(variations)


In [ ]:
clinvar_data = pd.read_csv('C:\\Users\\Kajus\\Desktop\\clinvar_result.txt', sep='\t')

display(clinvar_data)